In [1]:
import numpy as np # linear algebra
np.random.seed(666)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from subprocess import check_output

In [2]:
#Load data
train = pd.read_json("/mnt/extDisk/courses/data/statoil/data/train_half/train.json")
train_final = pd.read_json("/mnt/extDisk/courses/data/statoil/data/train_half/train_final.json")
valid = pd.read_json("/mnt/extDisk/courses/data/statoil/data/train_half/valid.json")
test = pd.read_json("/mnt/extDisk/courses/data/statoil/data/train/test.json")

train = pd.concat([train,train_final])

train.inc_angle = train.inc_angle.replace('na', 0)
train.inc_angle = train.inc_angle.astype(float).fillna(0.0)
valid.inc_angle = valid.inc_angle.replace('na', 0)
valid.inc_angle = valid.inc_angle.astype(float).fillna(0.0)
print("done!")

done!


In [7]:


# Train data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , (((x_band1+x_band1)/2))[:, :, :, np.newaxis]], axis=-1)
X_angle_train = np.array(train.inc_angle)
y_train = np.array(train["is_iceberg"])

# Valid data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in valid["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in valid["band_2"]])
X_valid = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , (((x_band1+x_band1)/2))[:, :, :, np.newaxis]], axis=-1)
X_angle_valid = np.array(valid.inc_angle)
y_valid = np.array(valid["is_iceberg"])

# Test data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_test = np.array(test.inc_angle)



In [4]:
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, merge
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
    
def get_model():
    
    
    kernel_size = (5,5)
    bn_model = 0
    p_activation = "elu"
    input_1 = Input(shape=(75, 75, 3), name="X_1")
    
    ## input CNN
    input_CNN  = BatchNormalization(momentum = 0.99)(input_1 )
    input_CNN  = Conv2D(32,kernel_size, activation=p_activation, padding='same') (input_CNN )
    input_CNN  = BatchNormalization(momentum = 0.99)(input_CNN )
    input_CNN  = MaxPooling2D((2,2)) (input_CNN )
    input_CNN  = Dropout(0.25)(input_CNN )
    input_CNN  = Conv2D(64,kernel_size, activation=p_activation, padding='same') (input_CNN )
    input_CNN  = BatchNormalization(momentum = 0.99)(input_CNN )
    input_CNN  = MaxPooling2D((2,2)) (input_CNN )
    input_CNN  = Dropout(0.25)(input_CNN )
    

    ## first residual
    
    input_CNN_residual  = BatchNormalization(momentum = 0.99)(input_CNN)
    input_CNN_residual  = Conv2D(128,kernel_size, activation=p_activation, padding='same') (input_CNN_residual )
    input_CNN_residual  = BatchNormalization(momentum = 0.99)(input_CNN_residual)
    input_CNN_residual  = Dropout(0.25)(input_CNN_residual )
    input_CNN_residual  = Conv2D(64,kernel_size, activation=p_activation, padding='same') (input_CNN_residual )
    input_CNN_residual  = BatchNormalization(momentum = 0.99)(input_CNN_residual)
    
    input_CNN_residual = merge([input_CNN_residual,input_CNN], mode = 'sum')
    
    ## final CNN
    
    top_CNN  = Conv2D(128,kernel_size, activation=p_activation, padding='same') (input_CNN_residual )
    top_CNN  = BatchNormalization(momentum = 0.99)(top_CNN)
    top_CNN  = MaxPooling2D((2,2)) (top_CNN )
    top_CNN  = Conv2D(256,kernel_size, activation=p_activation, padding='same') (top_CNN )
    top_CNN  = BatchNormalization(momentum = 0.99)(top_CNN)
    top_CNN  = Dropout(0.25)(top_CNN )
    top_CNN  = MaxPooling2D((2,2)) (top_CNN )
    top_CNN  = Conv2D(512,kernel_size, activation=p_activation, padding='same') (top_CNN )
    top_CNN  = BatchNormalization(momentum = 0.99)(top_CNN)
    top_CNN  = Dropout(0.25)(top_CNN )
    top_CNN  = MaxPooling2D((2,2)) (top_CNN )
    top_CNN  = GlobalMaxPooling2D() (top_CNN)
    

    
    layer_dense = Dense(512)(top_CNN)
    layer_dense  = BatchNormalization(momentum = 0.99)(layer_dense)
    layer_dense  = Dropout(0.5)(layer_dense )
    layer_dense = Dense(256)(layer_dense)
    layer_dense  = BatchNormalization(momentum = 0.99)(layer_dense)
    layer_dense  = Dropout(0.5)(layer_dense )
    output = Dense(1, activation="sigmoid")(layer_dense)
    
    
    model = Model(input_1,  output)
    optimizer = Adam(lr=0.001)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

model = get_model()
model.summary()

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:46: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/user/tensorflow/local/lib/python2.7/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
X_1 (InputLayer)                (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 75, 75, 3)    12          X_1[0][0]                        
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 75, 75, 32)   2432        batch_normalization_1[0][0]      
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 75, 75, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [5]:
file_path = "/mnt/extDisk/courses/data/statoil/data/models/.resnet.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=10)

model = get_model()
model.fit(X_train, y_train, epochs=35
          , validation_data=(X_valid, y_valid)
         , batch_size=32
         , callbacks=callbacks)


/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:46: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Train on 1404 samples, validate on 200 samples
Epoch 1/35
1404/1404 [==============================] - 4s 3ms/step - loss: 0.8682 - acc: 0.5812 - val_loss: 7.2531 - val_acc: 0.5500
Epoch 2/35
1404/1404 [==============================] - 2s 2ms/step - loss: 0.5552 - acc: 0.7279 - val_loss: 2.3237 - val_acc: 0.5500
Epoch 3/35
1404/1404 [==============================] - 2s 1ms/step - loss: 0.5055 - acc: 0.7635 - val_loss: 1.4859 - val_acc: 0.5950
Epoch 4/35
1404/1404 [==============================] - 2s 1ms/step - loss: 0.4245 - acc: 0.8063 - val_loss: 0.6900 - val_acc: 0.6900
Epoch 5/35
1404/1404 [==============================] - 2s 1ms/step - loss: 0.3585 - acc: 0.8433 - val_loss: 0.2977 - val_acc: 0.8750
Epoch 6/35
1404/1404 [==============================] - 2s 1ms/step - loss: 0.3182 - acc: 0.8611 - val_loss: 0.6863 - val_acc: 0.7600
Epoch 7/35
1404/1404 [==============================] - 2s 1ms/step - loss: 0.2697 - acc: 0.8846 - val_loss: 0.6528 - val_acc: 0.7350
Epoch 8/35
1404

In [6]:
model.load_weights(filepath=file_path)

print("Train evaluate:")
print(model.evaluate(X_train, y_train, verbose=1, batch_size=200))
print("####################")
print("watch list evaluate:")
print(model.evaluate(X_valid, y_valid, verbose=1, batch_size=200))

Train evaluate:
1404/1404 [==============================] - 1s 383us/step
[0.13577443256210059, 0.95085467979778926]
####################
watch list evaluate:
200/200 [==============================] - 0s 256us/step
[0.2824113667011261, 0.87999999523162842]


In [8]:
prediction = model.predict(X_test, verbose=1, batch_size=200)

8424/8424 [==============================] - 2s 292us/step


In [9]:
submission = pd.DataFrame({'id': test["id"], 'is_iceberg': prediction.reshape((prediction.shape[0]))})
submission.head(10)

,id,is_iceberg
0,5941774d,0.041207
1,4023181e,0.909202
2,b20200e4,0.125139
3,e7f018bb,0.999282
4,4371c8c3,0.389556
5,a8d9b1fd,0.814050
6,29e7727e,0.053633
7,92a51ffb,0.999693
8,c769ac97,0.021874
9,aee0547d,0.004081


In [10]:
submission.to_csv("/mnt/extDisk/courses/data/statoil/data/resnet/resnet_full.csv", index=False)